In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

from tqdm import tqdm

In [ ]:
import os

In [ ]:
test_predictions_folder = '../data/predictions/'

In [ ]:
from skimage.morphology import label

# https://www.kaggle.com/stainsby/fast-tested-rle-and-input-routines
def run_length_encode(mask):
    pixels = mask.T.flatten()
    # We need to allow for cases where there is a '1' at either end of the sequence.
    # We do this by padding with a zero at each end when needed.
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    rle = ' '.join(str(x) for x in rle)
    return rle

# https://www.kaggle.com/stainsby/fast-tested-rle-and-input-routines
def run_length_decode(rle_str, mask_shape, mask_dtype):
    s = rle_str.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    mask = np.zeros(np.prod(mask_shape), dtype=mask_dtype)
    for lo, hi in zip(starts, ends):
        mask[lo:hi] = 1
    return mask.reshape(mask_shape[::-1]).T


def probas_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield run_length_encode(lab_img == i)

def get_submit_data(predictions, ids):
    new_ids = []
    rles = []
    for n, id_ in enumerate(ids):
        rle = list(probas_to_rles(predictions[n]))
        if len(rle) == 0:
            rle.append('')
        rles.extend(rle)
        new_ids.extend([id_] * len(rle))
    return new_ids, rles

In [ ]:
predictions = []
test_img_ids = []

for test_img_file in os.listdir(test_predictions_folder):
    if test_img_file.endswith('.png'):
        test_img_id = test_img_file[:-4]
        test_img = plt.imread(os.path.join(test_predictions_folder, test_img_file))

        test_img_ids.append(test_img_id)
        predictions.append(test_img)

In [ ]:
new_ids, rles = get_submit_data(predictions, test_img_ids)

In [ ]:
submission = pd.DataFrame()
submission['ImageId'] = new_ids
submission['EncodedPixels'] = pd.Series(rles)
submission.to_csv('../submits/submission_retina_unet_001.csv', index=False)